In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df1= pd.read_csv("/kaggle/input/cheapest-electric-cars/Cheapestelectriccars-EVDatabase.csv")

In [4]:
df1.head()

In [5]:
print(df1.columns.tolist())

### Заполним пустые значения 0

In [6]:
df1=df1.fillna('0')

In [7]:
df1['Manufacturer'] = df1.Name.str.split(' ', 1, expand=True)[0]

## Отчистим данные
### Удалим знак валюты

In [8]:
PriceinUK=[]
for item in df1['PriceinUK']:
    PriceinUK+=[int(item.replace('£','').replace(',',''))]
df1['PriceinUK']=PriceinUK

In [9]:
PriceinGermany=[]
for item in df1['PriceinGermany']:
    PriceinGermany+=[int(item.replace('€','').replace(',',''))]
df1['PriceinGermany']=PriceinGermany

In [10]:
FastChargeSpeed=[]
for item in df1['FastChargeSpeed']:
    FastChargeSpeed+=[int(item.replace(' km/h','').replace('-','0'))]
df1['FastChargeSpeed']=FastChargeSpeed

In [11]:
Efficiency=[]
for item in df1['Efficiency']:
    Efficiency+=[int(item.replace(' Wh/km',''))]
df1['Efficiency']=Efficiency

In [12]:
Range=[]
for item in df1['Range']:
    Range+=[int(item.replace(' km',''))]
df1['Range']=Range

In [13]:
TopSpeed=[]
for item in df1['TopSpeed']:
    TopSpeed+=[int(item.replace(' km/h',''))]
df1['TopSpeed']=TopSpeed

In [14]:
Acceleration=[]
for item in df1['Acceleration']:
    Acceleration+=[float(item.replace(' sec',''))]
df1['Acceleration']=Acceleration

In [15]:
Subtitle=[]
for item in df1['Subtitle']:
    Subtitle+=[float(item.replace('Battery Electric Vehicle | ','').replace(' kWh','').replace('      ',''))]
df1['Subtitle']=Subtitle

In [16]:
df1= df1.rename(columns = {'Subtitle':'KWH'})

In [17]:
df1.head()

In [18]:
df1.info()

In [19]:
df1.corr()

In [20]:
plt.figure(figsize=(8,6))
sns.heatmap(df1.corr(), annot=True)

In [21]:
sns.countplot(x = 'Drive', data = df1)

In [22]:
sns.countplot(x = 'NumberofSeats', data = df1)

In [23]:
plt.figure(figsize=(8,6))
sns.countplot(x = 'NumberofSeats', hue='Drive', data=df1)

In [24]:
plt.figure(figsize=(18,10))
sns.countplot(y = 'Manufacturer', data = df1)

In [60]:
sns.catplot(data=df1, kind="bar", x="NumberofSeats", y="KWH",height=6, hue="Drive")
plt.title("КВТ/ч Т\С по сравнению с количеством мест и типом привода")
plt.show()

In [26]:
sns.relplot(x="KWH", y="Acceleration", height=6,hue="Drive",data=df1)

In [27]:
sns.relplot(x="KWH", y="Acceleration", size="NumberofSeats", height=6,sizes=(15, 100),data=df1)

In [28]:
sns.relplot(x="TopSpeed", y="Range",height=6, hue="Drive",data=df1)

In [29]:
sns.displot(
    df1, x="TopSpeed", col="Drive", 
    binwidth=3, height=7, facet_kws=dict(margin_titles=True),
)

In [30]:
sns.displot(
    df1, x="TopSpeed", col="NumberofSeats", 
    binwidth=3, height=7, facet_kws=dict(margin_titles=True),
)

### Корреляция между ценой и характеристиками Germany

In [61]:
sns.relplot(x="PriceinGermany", y="TopSpeed", height=6,data=df1)

### Корреляция между ценой и характеристиками UK

In [62]:
sns.relplot(x="PriceinUK", y="TopSpeed", height=6,data=df1)

In [33]:
sns.jointplot(x=df1["KWH"], y=df1["Range"], kind="hex", color="#4CB391")

In [34]:
sns.jointplot(x=df1["KWH"], y=df1["Efficiency"], kind="hex", color="#4CB391")

In [35]:
sns.relplot(x="FastChargeSpeed", y="Efficiency", height=6,data=df1)

In [36]:
sns.pairplot(df1[["KWH","Acceleration","TopSpeed","Range","FastChargeSpeed"]])

In [37]:
df1.head()

# Модель машинного обучения

Буду предсказывать цену Т\С в фунтах, т.к в евро
присудствует большое количество пропусков.

Учитывая, что 1 Евро = 0,85 Фунта.


In [63]:
c=0
for i in range(0, len(df1["PriceinUK"])):
    if (df1["PriceinUK"][i]==0):
        c+=1
print('Empty Values:',c)        

In [64]:
 pd.options.mode.chained_assignment = None

In [65]:
for i in range(0, len(df1["PriceinUK"])):
    if (df1["PriceinUK"][i]==0):
        val=df1["PriceinGermany"][i]*0.85
        df1["PriceinUK"][i]=val
        

In [66]:
df1.head()

In [67]:
import category_encoders as ce

train_df=df1

# create object of Ordinalencoding
encoder= ce.OrdinalEncoder(cols=['Drive'],return_df=True,
                           mapping=[{'col':'Drive',
'mapping':{'Front Wheel Drive':1,'Rear Wheel Drive':2,'All Wheel Drive':3}}])


#fit and transform train data 
df_train = encoder.fit_transform(train_df)#Original data
df_train.head()

In [68]:
df_train.info()

In [69]:
X= df_train.drop(['Name', 'PriceinGermany','PriceinUK','Manufacturer'], axis=1)
X=X.values

In [70]:
X

In [71]:
y=df_train['PriceinUK'].values

In [72]:
X.shape

In [73]:
y.shape

In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

In [75]:
from sklearn.ensemble import RandomForestRegressor
# create regressor object
regressor = RandomForestRegressor(n_estimators = 300, random_state = 0)

In [76]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

In [77]:
y_pred= regressor.predict(X_test)

In [78]:
y_test

In [79]:
y_pred

# Посчитаем ошибки моделей

In [80]:
#Mean Absolute Error(MAE)

from sklearn.metrics import mean_absolute_error
print("MAE: ",mean_absolute_error(y_test,y_pred))

In [81]:
#Mean Squared Error(MSE)

from sklearn.metrics import mean_squared_error
print("MSE: ",mean_squared_error(y_test,y_pred))

In [82]:
#Root Mean Squared Error(RMSE)

print("RMSE: ",np.sqrt(mean_squared_error(y_test,y_pred)))

In [83]:
#R Squared (R2)

from sklearn.metrics import r2_score
r2 = r2_score(y_test,y_pred)
print("R2: ",r2)